# 4 Working with Ethereum-ETL data
So far we have looked at interactive examples, where we've requested data from an Ethereum node. In large scale studies, this is too slow. For these, Ethereum-ETL provides a viable alternative. In this exercise, we will briefly look at how to work with CSV data generated from Ethereum-ETL.

# 4.1 Dataset
In the `data` directory, there are multiple CSV files containing information on blocks and transactions.
Let's assume they were generated by the following command:

```
ethereumetl export_blocks_and_transactions --start-block 12000000 --end-block 12000099 \
--provider-uri "http(s)://your-jsonrpc-endpoint-url(:port)" \
--blocks-output blocks12000000-12000099.csv --transactions-output transactions12000000-12000099.csv

ethereumetl extract_csv_column --input transactions12000000-12000099.csv --column hash --output transaction_hashes12000000-12000099.txt

ethereumetl export_receipts_and_logs --transaction-hashes transaction_hashes12000000-12000099.txt --provider-uri "http(s)://your-jsonrpc-endpoint-url(:port)" --receipts-output receipts12000000-12000099.csv --logs-output logs12000000-12000099.csv
```
And let's further assume we've done this 10 times, to cover the range of blocks 12000000-12000999 - 1000 blocks in total, in chunks of 100 each. Note that in reality you would want to choose a larger size...
The point is, there are several CSV files to deal with.

These consist of:
- blocks
- transactions
- receipts
- logs

# 4.2 Reading multiple files with Dask
We will be using Dask to read multiple files and work with the dataset *out of core* - this means even if we don't have enough memory to fit everything, we can still proceed.

Dask dataframes work very similar to pandas dataframes, but they are lazy. You can transform the data, but only when you run `.compute()` or `.head()`, the result will actually be computed.

In [1]:
import dask.dataframe as dd
import pandas as pd

# And we'll register a progress bar:
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [2]:
dd.read_csv("Ethereum-Data/block*").compute()

[########################################] | 100% Completed |  0.1s


,number,hash,parent_hash,nonce,sha3_uncles,logs_bloom,transactions_root,state_root,receipts_root,miner,difficulty,total_difficulty,size,extra_data,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas
0,12000000,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,0x5855b177bf334278228bb6dc8c7be272d6828ffce8ba...,0xb62c052c3d4a3866,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x40a40328609aa01d1514a588f001962842101a449091...,0x1b1f04a1ec5c282c78e5b83f3fd947cfc6fb61677e0d...,0x8da999d414392e86fda9a3bc48a1fb35dcb8bbfef6c3...,0x0bdbc8f25e31e43cf5b2ff325711f7bb06ccea31a53a...,0xb3b7874f13387d44a3398d298b075b7a3505d8d4,5648642301448524,21833228890297050240453,47759,0x426162656c20687a38,12506011,12490032,1615234816,263,NaN
1,12000001,0x4bd35cb48395e77fd317a0309342c95d6687dbc4fcb8...,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,0x7f04dcf9f566310b,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0xb2625924548a8021b584a0e0804114e3504a1a138c08...,0x7130a1c570c869bce37cffb6bb23614923ec2db17f32...,0xd17974c8152602f2dae57140027df110f2495e66b2e2...,0x994a9a7cf1aa6f02bba3600995d5f8bbe994388aa521...,0xea674fdde714fd979de3edf0f56aa9716b898ec8,5645884443760226,21833234536181494000679,55644,0x65746865726d696e652d617369612d6561737432,12493800,12487154,1615234841,170,NaN
2,12000002,0xf67b3723d31f878f0dd9dc7ab7b633f6176d58106723...,0x4bd35cb48395e77fd317a0309342c95d6687dbc4fcb8...,0x4cf091579ef3d32a,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x112851e040138048f189a521a9035c801b9293941510...,0xef797d564aff8e4ba8503d1d47553841b076bed4934e...,0xce62d6b4f15b225103af312be33eb718fa452f2d0696...,0xe479f9f0bc9824ac9f21cc1b18e4849809f6352e8cf5...,0x04668ec2f57cc15c381b461b9fedab5d451c8f7f,5645884712195682,21833240182066206196361,47694,0x73706964657230380467cd74,12499892,12488406,1615234854,213,NaN
3,12000003,0x43ae52f8916d7c671d7e1fea552888717b0389ebf3cd...,0xf67b3723d31f878f0dd9dc7ab7b633f6176d58106723...,0x13b362a7c4f3ffc5,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x9ca671285506204b9ce2b3b8fe835d25504881f08744...,0xefc034e1f87aa0122c6f7093632ed219fd581f392a49...,0xcf9c918b3c12c35b637b9c21156991b2ff1689c826d8...,0x077c2d6d86e6f1ca38172df74d963da04f721778b03e...,0x1ad91ee08f21be3de0ba2ba6918e714da6b45836,5637614641697260,21833245819680847893621,52077,0x30783438363937363635366636653230353036663666...,12500000,12479152,1615234896,162,NaN
4,12000004,0x97c266ba6427b1b33f7b4bc6924709891ce729badbe2...,0x43ae52f8916d7c671d7e1fea552888717b0389ebf3cd...,0x566be903bd5e3a21,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x93f055ac5a829212bb182e02c0097e04002214501540...,0xb378a70671be9fbb9ca355f3f2d1926f0319970b1e72...,0x149903185b6cb3350ba0a8600913187738ea463abec1...,0x852eb5ee9aac7aeaf243f15448a90831da1265ec7dbf...,0xea674fdde714fd979de3edf0f56aa9716b898ec8,5637614910132716,21833251457295758026337,64692,0x65746865726d696e652d75732d6561737431,12487794,12476155,1615234906,248,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,12001071,0xab28344a6c94a088c48d3bd6d3bc10e43caf129eacdd...,0x405f27c06bf099b73fd34c77a3cd760835b6616f391f...,0xd84b888a626a76c4,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x06a011d1a041afc193ecbc90ba7f16946a70061ab319...,0xbc8c9f74d18e2f94b6b7000cc90b9769b65539a95a86...,0x5c0ae50c9d4cd145a45c35855b431c556f6dc66231e2...,0xa0348b2091f64f62b1ff85d7b93a5f4503b47f8651e2...,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,5554453679204305,21839228475506190413013,46498,0x6e616e6f706f6f6c2e6f7267,12463383,12449341,1615249397,158,NaN
96,12001024,0xba6662b0d594385baac2fb6cfd764753a4b5d45dc009...,0xfe5db42b0891d9bc2ce58013ed0a685964facd2a3e92...,0xf1f63f2caee7625d,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0xfc62e4c3c6f2d15694ac82a0a801be23110a50095237...,0xe483f6f6c052483231e21a2c9d2c9c5560ea09823239...,0x077186eb261f3c7ac6b3912da8601d4bc01916f59397...,0xde0569ced35388f05a053e9435f2df083e0cf7b5299c...,0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c,5559918404040134,218389669153195551

In [3]:
logsDD = dd.read_csv("Ethereum-Data/logs*")
logsDD.head()

[########################################] | 100% Completed |  0.2s


,log_index,transaction_hash,transaction_index,block_hash,block_number,address,data,topics
0,99,0xf8b86de64ecb735d3e0e3dfb0c0a34f6de6e64b83cdc...,101,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,12000000,0xf4cd3d3fda8d7fd6c5a500203e38640a70bf9577,0x00000000000000000000000000000000000000000000...,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...
1,100,0x9a4ff3c9c38d7d8e67f30654e049913fd201394e4e91...,103,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,12000000,0x55652ce84d686177c8946e8c78078c0d6cfa4b30,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
2,101,0x2f31f5065b2c685f739c1ef0ec4fb6bfb63c6d82e260...,106,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,12000000,0xdac17f958d2ee523a2206206994597c13d831ec7,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
3,102,0x4c69f768566e295087734003b58e38cf2faa0ba7cc96...,107,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,12000000,0xf4cd3d3fda8d7fd6c5a500203e38640a70bf9577,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
4,103,0x4c69f768566e295087734003b58e38cf2faa0ba7cc96...,107,0x3c9c46a46b17361cd1ac3ed3401c9a268095c1810bf9...,12000000,0xf4cd3d3fda8d7fd6c5a500203e38640a70bf9577,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...


# 4.3 Extracting token transfers from logs
We can't easily rely on parsing logs with web3py, but we can look at the logs individually

In [4]:
from web3 import Web3

transferTopic = Web3.keccak(text='Transfer(address,address,uint256)').hex()

# We filter all log topics to those that start with the keccak hash of the transfer topic
# and which consist of 3 values, the topic hash, the from and to address, but not the transferred token value
# as the latter is in the data column
logsDD["topicLen"] = logsDD.topics.apply(lambda x: len(str(x).split(",")), meta=('topics', 'int8'))
tokenTransfersDD = logsDD[logsDD.topics.str.startswith(transferTopic, na=False) & (logsDD.topicLen == 3)]

# 4.4 Utility functions
We now define a couple of utility functions to convert hex to int, and extract addresses and numbers

In [5]:
def hexToInt(hexNumberString):
    return(str(int(hexNumberString, 16)))

def extractAddress(string64):
    return("0x"+string64[-40:])

def parseField(dat, number, convert="number"):
    theSlice = dat[(2+number*64):(2+(number+1)*64)]
    if(convert == 'number'):
        theSlice = hexToInt(theSlice)
    elif(convert == 'address'):
        theSlice = extractAddress(theSlice)
    return(theSlice)

# 4.5 Extracting from, to and value of the token transfers
Now we are ready to define how the from and to address of the token transfers should be extracted, and we will get the transferred amount from the data column. Finally, we select which columns we want and compute!

The result is then a pandas dataframe of token transfers!

In [6]:
tokenTransfersDD["from"] = tokenTransfersDD.topics.apply(lambda x: extractAddress(x.split(",")[1]), meta=('from', object))
tokenTransfersDD["to"] = tokenTransfersDD.topics.apply(lambda x: extractAddress(x.split(",")[2]), meta=('to', object))
tokenTransfersDD["value"] = tokenTransfersDD.data.apply(lambda x: parseField(x, 0, "number"), meta=("value", object))

In [7]:
tokenTransfersDF = tokenTransfersDD[["address","from","to","value"]].compute()

[########################################] | 100% Completed |  2.2s


In [8]:
tokenTransfersDF

,address,from,to,value
1,0x55652ce84d686177c8946e8c78078c0d6cfa4b30,0xce0a5945a2e7163568221186da084ad34edad14f,0xe52dee5d5821090d2acaf5f9c136d18d2a9d120b,50000000000000000000
2,0xdac17f958d2ee523a2206206994597c13d831ec7,0x0785db9cebaf421fa394052c3d4d4432c1303b2f,0x84618695f59dec51f5cc5b2ea487935f226cff13,20000000000
3,0xf4cd3d3fda8d7fd6c5a500203e38640a70bf9577,0x8f3b188b0ebbb920908b206aeb83a9eed10cf187,0x4599cda238fb71573fd5a0076c199320e09bcff0,656255009787569701
4,0xf4cd3d3fda8d7fd6c5a500203e38640a70bf9577,0x4599cda238fb71573fd5a0076c199320e09bcff0,0x8f3b188b0ebbb920908b206aeb83a9eed10cf187,84761921947506338
6,0xf4cd3d3fda8d7fd6c5a500203e38640a70bf9577,0x4599cda238fb71573fd5a0076c199320e09bcff0,0xf29b6cfd7019df58267fbcba3c0915f66baca08e,9843825146813545
...,...,...,...,...
30402,0x7a5ce6abd131ea6b148a022cb76fc180ae3315a6,0xc4ec5f5bb2f3a887a86e2c9990b73ec2562febbb,0xc75253291c312e9edd3bbeb5cc228eb50f2e2950,1819255545531257000
30404,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc75253291c312e9edd3bbeb5cc228eb50f2e2950,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,5041753169919230960
30409,0xdac17f958d2ee523a2206206994597c13d831ec7,0x0f57f673ace6f00b5464d013f6cd244b65a492c7,0x39f6a6c85d39d5abad8a398310c52e7c374f2ba3,17000000
30410,0xfad45e47083e4607302aa43c65fb3106f1cd7607,0x39f6a6c85d39d5abad8a398310c52e7c374f2ba3,0xe56472461a93868b9edeec3552b32d90bbf0464c,2054252480000000
